In [1]:
import numpy as np
import pandas as pd
import time
%load_ext line_profiler

In [2]:
data = pd.read_csv("adult.data", delimiter=", ", engine='python', names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
                                                                         'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [3]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
temp = data[["age", "fnlwgt", "education-num", "income"]].copy()
temp["income"] = (temp.income==">50K").astype('int')
y = temp[["income"]].values
X = temp[["age", "fnlwgt", "education-num"]].copy().to_numpy()

## Algorithm

In [5]:
def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

In [6]:
def loss(X, y, w):
    margin = np.dot(X, w)
    
    return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))

In [7]:
def gradients(X, y, y_hat):
    m = X.shape[0]
    
    # Gradient of loss w.r.t weights
    dw = (1/m)*np.dot(X.T, (y_hat - y))
    
    # Gradient of loss w.r.t bias
    db = (1/m)*np.sum((y_hat - y)) 
    
    return dw, db


In [8]:
def normalize(X): 
    m, n = X.shape

    for i in range(n):
        X = (X - X.mean(axis=0))/X.std(axis=0)
        
    return X

In [9]:
def train(X, y, bs, epochs, lr):
    m, n = X.shape
    
    # Initializing weights and bias to zeros.
    w = np.zeros((n,1))
    b = 0
    
    # Reshape y.
    y = y.reshape(m,1)
    
    # Normalize inputs
    x = normalize(X)
    
    # Store losses
    losses = []
    
    # Train
    for epoch in range(epochs):
        for i in range((m-1)//bs + 1):
            
            # Defining batches for SGD (this can be changed)
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            # Predict
            y_hat = sigmoid(np.dot(xb, w) + b)
            
            # Calculate gradients
            dw, db = gradients(xb, yb, y_hat)
            
            # Update params
            w -= lr*dw
            b -= lr*db
        
        # Calc loss
        l = loss(x, y, w)
        losses.append(l)
        
    return w, b, losses

In [10]:
def predict(X):
    
    # X --> Input.
    
    # Normalizing the inputs.
    x = normalize(X)
    
    # Calculating presictions/y_hat.
    preds = sigmoid(np.dot(X, w) + b)
    
    # if y_hat >= 0.5 --> round up to 1
    # if y_hat < 0.5 --> round up to 1
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

In [11]:
def accuracy(y, y_hat):
    accuracy = np.sum(y == y_hat) / len(y)
    return accuracy

In [15]:
%lprun -f train w, b, l = train(X, y, bs=100, epochs=1000, lr=0.001)

<ipython-input-5-2837dfe2f958>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1 + np.exp(-z))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: overflow encountered in exp
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: invalid value encountered in multiply
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: invalid value encountered in logaddexp
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: overflow encountered in add
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))


In [12]:
%%time
# Training 
for i in range(10):
    w, b, l = train(X, y, bs=100, epochs=1000, lr=0.001)

<ipython-input-5-2837dfe2f958>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1 + np.exp(-z))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: overflow encountered in exp
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: invalid value encountered in multiply
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: invalid value encountered in logaddexp
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))
<ipython-input-6-446f1ee1a133>:4: RuntimeWarning: overflow encountered in add
  return y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin))))


Wall time: 5min 15s


In [13]:
print(f'weights: {w}, beta: {b}')

weights: [[476.68326575]
 [-49.8336871 ]
 [126.64328737]], beta: 0.49218514721669687


In [14]:
from logistic_regression import LogisticRegression

ModuleNotFoundError: No module named 'logistic_regression'

In [ ]:
lr = LogisticRegression(max_epochs=1000, learning_rate=0.001, batch_size=100)
lr.train(X, y)

In [ ]:
print(f'weights: {lr.get_weights()}, beta: {lr.get_bias()}')

## TO DO:

### Step 1: Clean (and slow) baseline

- Change code into singular class? (for presentability)

- Implement minibatch if it isn't already

- Add in regularization

- Replace np.sum(), np.dot(), np.mean(), etc. with handmade functions

- Implement data science pipeline (including grid search)


### Step 2: Speed up

Find different methods to improve baseline and implement (look through syllabus):

1) Use numpy

2) Cython

3) Parallel computing (esp. for gradient descent)

4) ...


#### Minor step 3: Get clean output


Timer unit: 1e-07 s

Total time: 43.397 s
File: <ipython-input-9-a765c3261458>
Function: train at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def train(X, y, bs, epochs, lr):
     2         1        105.0    105.0      0.0      m, n = X.shape
     3                                               
     4                                               # Initializing weights and bias to zeros.
     5         1        294.0    294.0      0.0      w = np.zeros((n,1))
     6         1         11.0     11.0      0.0      b = 0
     7                                               
     8                                               # Reshape y.
     9         1        113.0    113.0      0.0      y = y.reshape(m,1)
    10                                               
    11                                               # Normalize inputs
    12         1      83837.0  83837.0      0.0      x = normalize(X)
    13                                               
    14                                               # Store losses
    15         1         33.0     33.0      0.0      losses = []
    16                                               
    17                                               # Train
    18      1001      13813.0     13.8      0.0      for epoch in range(epochs):
    19    327000    3537322.0     10.8      0.8          for i in range((m-1)//bs + 1):
    20                                                       
    21                                                       # Defining batches for SGD (this can be changed)
    22    326000    4032589.0     12.4      0.9              start_i = i*bs
    23    326000    4141838.0     12.7      1.0              end_i = start_i + bs
    24    326000    6599091.0     20.2      1.5              xb = X[start_i:end_i]
    25    326000    5866080.0     18.0      1.4              yb = y[start_i:end_i]
    26                                                       
    27                                                       # Predict
    28    326000  139321947.0    427.4     32.1              y_hat = sigmoid(np.dot(xb, w) + b)
    29                                                       
    30                                                       # Calculate gradients
    31    326000  156206727.0    479.2     36.0              dw, db = gradients(xb, yb, y_hat)
    32                                                       
    33                                                       # Update params
    34    326000   24956511.0     76.6      5.8              w -= lr*dw
    35    326000    6578424.0     20.2      1.5              b -= lr*db
    36                                                   
    37                                                   # Calc loss
    38      1000   82587821.0  82587.8     19.0          l = loss(x, y, w)
    39      1000      43858.0     43.9      0.0          losses.append(l)
    40                                                   
    41         1         14.0     14.0      0.0      return w, b, losses